# Асинхронное программирование

## Что это такое?

Допустим вы готовите завтрак. Вам нужно сварить кофе, пожарить яичницу, сделать бутерброд. В синхронной манере вы будете
делать примерно следующее:
1. Сделать яичницу
2. Сделать бутерброд
3. Сделать кофе
Вы не начнете готовить кофе пока не будет готов бутерброд. Кажется нелепым, зачем ждать когда пожарится яичница, если она
и без вашего надзора хорошо это делает. 

В случае асинхронного программирования вы ставите жариться яичницу, варите кофе и пока они готовятся делаете бутерброд. 
Времени тратите меньше, а делаете ту же работу.

## Приготовим завтрак синхронно

In [5]:
import time

def make_breakfast_slow():
    make_fried_eggs_slow()
    make_sandwich_slow()
    make_coffee_slow()

def make_fried_eggs_slow():
    time.sleep(2)
    print("Fried eggs are ready!")

def make_sandwich_slow():
    time.sleep(1)
    print("Sandwich is ready!")

def make_coffee_slow():
    time.sleep(1)
    print("Coffee is ready!")

make_breakfast_slow()

Fried eggs are ready!
Sandwich is ready!
Coffee is ready!


## Приготовим завтрак асинхронно

In [4]:
import asyncio

async def make_fried_eggs_fast():
    await asyncio.sleep(2)
    print("Fried eggs are ready!")

async def make_sandwich_fast():
    await asyncio.sleep(1)
    print("Sandwich is ready!")

async def make_coffee_fast():
    await asyncio.sleep(1)
    print("Coffee is ready!")

async def make_breakfast_fast():
    eggs_task = asyncio.create_task(make_fried_eggs_fast())
    coffee_task = asyncio.create_task(make_coffee_fast())
    await make_sandwich_fast()
    await asyncio.gather(eggs_task, coffee_task)

await make_breakfast_fast()


Sandwich is ready!
Coffee is ready!
Fried eggs are ready!


### Что произошло? 
1. Мы определили асинхронные функции с помощью `async`. Внутри функций мы используем неблокирующий `asyncio.sleep` 
и ждем его с помощью `await`. Это говорит интерпретатору: "можешь идти работать дальше, а потом как отработаю я сообщу тебе".
2. Мы запустили в работу сразу две функции по готовке кофе и яичницы, обернув их в `task`. Встретив `await` интерпретатор сразу вышел из них и пошел работать дальше
3. Встретил `await make_sandwich()`. Тут ему делать нечего, так как больше нечего ему обрабатывать и он спокойно ждет пока будет готов бутерброд
4. Затем мы ждем завершения работы обоих функций по готовке яиц и кофе с помощью `await asyncio.gather(eggs_task, coffee_task)`

## Как все устроено

Посмотрим на типы функций и возвращаемых значений

In [7]:
print(type(make_breakfast_slow))
print(type(make_breakfast_slow()))

<class 'function'>
Fried eggs are ready!
Sandwich is ready!
Coffee is ready!
<class 'NoneType'>


In [8]:
print(type(make_breakfast_fast))
print(type(make_breakfast_fast()))

<class 'function'>
<class 'coroutine'>


/var/folders/fm/k78pgbtj1v118606k1hh97240000gn/T/ipykernel_50792/1525278321.py:2: RuntimeWarning: coroutine 'make_breakfast_fast' was never awaited
  print(type(make_breakfast_fast()))


Не обращаем внимания на предупреждение. Тип функции это функция, а вот возвращает функция не `None`, а некую корутину. 

# Попробуем в деле

Начнем с синхронных запросов

Просто установим пакет для отправки HTTP запросов

In [11]:
%pip install requests
%pip install python-dotenv
%pip install aiohttp


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [aiohttp]m7/8 [aiohttp]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
import os
from dotenv import load_dotenv 

load_dotenv()
API_KEY = os.getenv("API_KEY")

def get_weather(city):
    url = f'http://api.openweathermap.org/data/2.5/weather'
    params = {'q': city, 'APPID': API_KEY}

    weather_json = requests.get(url=url, params=params).json()
    print(f'{city}: {weather_json["weather"][0]["main"]}')

def main(cities_):
    for city in cities_:
        get_weather(city)


cities = ['Moscow', 'St. Petersburg', 'Rostov-on-Don', 'Kaliningrad', 'Vladivostok',
          'Minsk', 'Beijing', 'Delhi', 'Istanbul', 'Tokyo', 'London', 'New York']

main(cities)

Moscow: Clouds
St. Petersburg: Clouds
Rostov-on-Don: Clouds
Kaliningrad: Clouds
Vladivostok: Clouds
Minsk: Clouds
Beijing: Clouds
Delhi: Haze
Istanbul: Clouds
Tokyo: Mist
London: Clouds
New York: Clear


Теперь сделаем то же самое, но асинхронно

In [10]:
import asyncio
from aiohttp import ClientSession


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': API_KEY}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            return f'{city}: {weather_json["weather"][0]["main"]}'


async def main(cities_):
    tasks = []
    for city in cities_:
        tasks.append(asyncio.create_task(get_weather(city)))

    results = await asyncio.gather(*tasks)

    for result in results:
        print(result)


cities = ['Moscow', 'St. Petersburg', 'Rostov-on-Don', 'Kaliningrad', 'Vladivostok',
          'Minsk', 'Beijing', 'Delhi', 'Istanbul', 'Tokyo', 'London', 'New York']

await main(cities)

Moscow: Clouds
St. Petersburg: Clouds
Rostov-on-Don: Clouds
Kaliningrad: Clouds
Vladivostok: Clouds
Minsk: Clouds
Beijing: Clouds
Delhi: Haze
Istanbul: Clouds
Tokyo: Clouds
London: Clouds
New York: Clear


## Источники
1. https://habr.com/ru/articles/667630/